In [1]:
# Another quick start example on how one might write a metric for a transient population

In [2]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import rubin_sim.maf as maf
import healpy as hp
from rubin_sim.data import get_baseline

In [3]:
# Need a database to work on. You can grab this one from: 
# http://astro-lsst-01.astro.washington.edu:8080/fbs_db/fbs_2.99/draft/draft_connected_v2.99_10yrs.db

# all the columns in the database documented at: https://rubin-sim.lsst.io/rs_scheduler/output_schema.html

# or just use the baseline that comes with rubin_sim
filename = get_baseline()
run_name = filename.replace('.db', '')


In [5]:
# very simple. Probably want to actually compute things per filter
def simple_light_curve(nights, peak_times, amplitude, duration, mag=22):
    result = nights*0 + mag
    boosted_indx = np.where((nights >= peak_times) & (nights <= (peak_times - duration)))[0]
    result[boosted_indx] -= amplitude  # negative since magnitudes are backwards
    
    return result

In [4]:
from rubin_sim.utils import uniform_sphere

def generate_transient_slicer(
    transient_amplitude=1,
    transient_duration=10,
    t_start=1,
    t_end=3652,
    n_events=10000,
    seed=42,
    nside=128,
    filtername="r",
):
    """
    Generate a UserPointSlicer with a population of transient events. To be used with

    Parameters
    ----------
    transient_amplitude : float (1)
        Some amplitude (mags)
    transient_duration : float (10)
        Duration of time for the events generated (days)
    t_start : float (1)
        The night to start generating peaks (days)
    t_end : float (3652)
        The night to end generating peaks (days)
    n_events : int (10000)
        Number of events to generate
    seed : float (42)
        Random number seed
    nside : int (128)
        HEALpix nside, used to pick which stellar density map to load
    filtername : str ('r')
        The filter to use for the stellar density map
    """
    np.random.seed(seed)

    # put points uniformly all over the sky
    # other metrics have examples of distributing like stars in the Galaxy
    ra, dec = uniform_sphere(n_events, seed=seed)
    
    # When the events happen
    peak_times = np.random.uniform(low=t_start, high=t_end, size=n_events)

    # Set up the slicer to evaluate the catalog we just made
    slicer = slicers.UserPointsSlicer(ra, dec, lat_lon_deg=True, badval=0)
    # Add any additional information about each object to the slicer
    slicer.slice_points["peak_time"] = peak_times
    slicer.slice_points["transient_amplitude"] = transient_amplitude
    slicer.slice_points["transient_duration"] = transient_duration

    return slicer


In [ ]:
# Let's make a metric
class SimpleTransientMetric(maf.BaseMetric):
    def __init__(self):
        pass

    def run(self, data_slice, slice_point=None):
        
        # Call the light curve function
        light_curve = simple_light_curve(data_slice[self.night_col],
                                         slice_point["peak_time"],
                                         slice_point["amplitude"],
                                         slice_point["duration"])
        
        # do we detect a change?
        
        
        pass
    